In [ ]:
# --- Analysis Notebook: All Queries in One Cell ---
import duckdb
import pandas as pd

# Connect to DuckDB
con = duckdb.connect('pipeline.duckdb')

# 1. Total Users
df_total_users = con.execute("SELECT COUNT(*) AS total_users FROM users;").fetchdf()
print("Total Users:\n", df_total_users, "\n")

# 2. Total Revenue
df_total_revenue = con.execute("SELECT SUM(total_amount) AS total_revenue FROM orders;").fetchdf()
print("Total Revenue:\n", df_total_revenue, "\n")

# 3. Top 10 Products by Revenue
df_top_products = con.execute("""
SELECT p.product_id, p.name, SUM(o.total_amount) AS revenue
FROM products p
JOIN orders o ON p.product_id = o.product_id
GROUP BY p.product_id, p.name
ORDER BY revenue DESC
LIMIT 10;
""").fetchdf()
print("Top 10 Products by Revenue:\n", df_top_products, "\n")

# 4. Top 10 Users by Spending
df_top_users = con.execute("""
SELECT u.user_id, u.name, SUM(o.total_amount) AS spend
FROM users u
JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
ORDER BY spend DESC
LIMIT 10;
""").fetchdf()
print("Top 10 Users by Spending:\n", df_top_users, "\n")

# 5. Monthly Revenue with Growth
df_monthly_revenue = con.execute("""
WITH monthly_revenue AS (
    SELECT DATE_TRUNC('month', order_date) AS month,
           SUM(total_amount) AS revenue
    FROM orders
    GROUP BY month
)
SELECT *,
       revenue - LAG(revenue) OVER (ORDER BY month) AS growth
FROM monthly_revenue;
""").fetchdf()
print("Monthly Revenue with Growth:\n", df_monthly_revenue, "\n")

# 6. Revenue by Product Category
df_category_revenue = con.execute("""
SELECT p.category, SUM(o.total_amount) AS revenue
FROM products p
JOIN orders o ON p.product_id = o.product_id
GROUP BY p.category
ORDER BY revenue DESC;
""").fetchdf()
print("Revenue by Product Category:\n", df_category_revenue, "\n")

# 7. Orders per User (Top 10)
df_orders_per_user = con.execute("""
SELECT u.user_id, u.name, COUNT(o.order_id) AS total_orders
FROM users u
JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
ORDER BY total_orders DESC
LIMIT 10;
""").fetchdf()
print("Orders per User (Top 10):\n", df_orders_per_user, "\n")

# 8. Average Order Value per User (Top 10)
df_avg_order_value = con.execute("""
SELECT u.user_id, u.name, AVG(o.total_amount) AS avg_order_value
FROM users u
JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
ORDER BY avg_order_value DESC
LIMIT 10;
""").fetchdf()
print("Average Order Value per User (Top 10):\n", df_avg_order_value, "\n")

# 9. Most Popular Product Category
df_popular_category = con.execute("""
SELECT p.category, COUNT(o.order_id) AS order_count
FROM products p
JOIN orders o ON p.product_id = o.product_id
GROUP BY p.category
ORDER BY order_count DESC;
""").fetchdf()
print("Most Popular Product Category:\n", df_popular_category, "\n")
